Suppose that your business relies on computing services where the power consumed by your machines varies throughout the day. You do not know the actual cost of the electricity consumed by the machines throughout the day, but the organization has provided you with historical data of the price of the electricity consumed by the machines.

1. DateTime: Date and time of the record
2. Holiday: contains the name of the holiday if the day is a national holiday
3. HolidayFlag: contains 1 if it’s a bank holiday otherwise 0
4. DayOfWeek: contains values between 0-6 where 0 is Monday
5. WeekOfYear: week of the year
6. Day: Day of the date
7. Month: Month of the date
8. Year: Year of the date
9. PeriodOfDay: half-hour period of the day
10. ForcastWindProduction: forecasted wind production
11. SystemLoadEA forecasted national load
12. SMPEA: forecasted price
13. ORKTemperature: actual temperature measured
14. ORKWindspeed: actual windspeed measured
15. CO2Intensity: actual C02 intensity for the electricity produced
16. ActualWindProduction: actual wind energy production
17. SystemLoadEP2: actual national system load
18. SMPEP2: the actual price of the electricity consumed (labels or values to be predicted)

train a machine learning model to predict the price of electricity consumed by the machines.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("electricity.csv", low_memory= False)
df.head()

,DateTime,Holiday,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay,ForecastWindProduction,SystemLoadEA,SMPEA,ORKTemperature,ORKWindspeed,CO2Intensity,ActualWindProduction,SystemLoadEP2,SMPEP2
0,01/11/2011 00:00,None,0,1,44,1,11,2011,0,315.31,3388.77,49.26,6.00,9.30,600.71,356.00,3159.60,54.32
1,01/11/2011 00:30,None,0,1,44,1,11,2011,1,321.80,3196.66,49.26,6.00,11.10,605.42,317.00,2973.01,54.23
2,01/11/2011 01:00,None,0,1,44,1,11,2011,2,328.57,3060.71,49.10,5.00,11.10,589.97,311.00,2834.00,54.23
3,01/11/2011 01:30,None,0,1,44,1,11,2011,3,335.60,2945.56,48.04,6.00,9.30,585.94,313.00,2725.99,53.47
4,01/11/2011 02:00,None,0,1,44,1,11,2011,4,342.90,2849.34,33.75,6.00,11.10,571.52,346.00,2655.64,39.87


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38014 entries, 0 to 38013
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   DateTime                38014 non-null  object
 1   Holiday                 38014 non-null  object
 2   HolidayFlag             38014 non-null  int64 
 3   DayOfWeek               38014 non-null  int64 
 4   WeekOfYear              38014 non-null  int64 
 5   Day                     38014 non-null  int64 
 6   Month                   38014 non-null  int64 
 7   Year                    38014 non-null  int64 
 8   PeriodOfDay             38014 non-null  int64 
 9   ForecastWindProduction  38014 non-null  object
 10  SystemLoadEA            38014 non-null  object
 11  SMPEA                   38014 non-null  object
 12  ORKTemperature          38014 non-null  object
 13  ORKWindspeed            38014 non-null  object
 14  CO2Intensity            38014 non-null  object
 15  Ac

In [4]:
df.describe()

,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay
count,38014.000000,38014.000000,38014.000000,38014.000000,38014.000000,38014.000000,38014.000000
mean,0.040406,2.997317,28.124586,15.739412,6.904246,2012.383859,23.501105
std,0.196912,1.999959,15.587575,8.804247,3.573696,0.624956,13.853108
min,0.000000,0.000000,1.000000,1.000000,1.000000,2011.000000,0.000000
25%,0.000000,1.000000,15.000000,8.000000,4.000000,2012.000000,12.000000
50%,0.000000,3.000000,29.000000,16.000000,7.000000,2012.000000,24.000000
75%,0.000000,5.000000,43.000000,23.000000,10.000000,2013.000000,35.750000
max,1.000000,6.000000,52.000000,31.000000,12.000000,2013.000000,47.000000


In [5]:
df.shape

(38014, 18)

In [6]:
# check for missing values
df.isna().sum()

DateTime                  0
Holiday                   0
HolidayFlag               0
DayOfWeek                 0
WeekOfYear                0
Day                       0
Month                     0
Year                      0
PeriodOfDay               0
ForecastWindProduction    0
SystemLoadEA              0
SMPEA                     0
ORKTemperature            0
ORKWindspeed              0
CO2Intensity              0
ActualWindProduction      0
SystemLoadEP2             0
SMPEP2                    0
dtype: int64

In [7]:
# view unique values of object dtype variables
def print_unique(df):
    for column in df:
        if df[column].dtypes == "object":
            print(f'{column}: {df[column].unique()}')

In [8]:
print_unique(df)

DateTime: ['01/11/2011 00:00' '01/11/2011 00:30' '01/11/2011 01:00' ...
 '31/12/2013 22:30' '31/12/2013 23:00' '31/12/2013 23:30']
Holiday: ['None' 'Christmas Eve' 'Christmas' "St Stephen's Day" "New Year's Eve"
 "New Year's Day" "St Patrick's Day" 'Good Friday' 'Holy Saturday'
 'Easter' 'Easter Monday' 'May Day' 'June Bank Holiday'
 'August Bank Holiday' 'October Bank Holiday']
ForecastWindProduction: ['315.31' '321.80' '328.57' ... '1152.01' '1123.67' '1094.24']
SystemLoadEA: ['3388.77' '3196.66' '3060.71' ... '3821.44' '3724.21' '3638.16']
SMPEA: ['49.26' '49.10' '48.04' ... '74.84' '149.77' '160.43']
ORKTemperature: ['6.00' '5.00' '4.00' '8.00' '9.00' '?' '11.00' '12.00' '13.00' '10.00'
 '7.00' '3.00' '14.00' '2.00' '1.00' '0.00' '-1.00' '15.00' '16.00'
 '17.00' '18.00' '19.00' '20.00' '21.00' '22.00' '-2.00' '-3.00' '-4.00'
 '23.00' '24.00' '25.00' '-0.00']
ORKWindspeed: ['9.30' '11.10' '7.40' '5.60' '3.70' '?' '13.00' '22.20' '18.50' '20.40'
 '24.10' '25.90' '27.80' '31.50' '29.6

some numerical values are recorded as object type

convert them to float

In [43]:
df.dtypes

DateTime                  object
Holiday                   object
HolidayFlag                int64
DayOfWeek                  int64
WeekOfYear                 int64
Day                        int64
Month                      int64
Year                       int64
PeriodOfDay                int64
ForecastWindProduction    object
SystemLoadEA              object
SMPEA                     object
ORKTemperature            object
ORKWindspeed              object
CO2Intensity              object
ActualWindProduction      object
SystemLoadEP2             object
SMPEP2                    object
dtype: object

In [44]:
import pandas as pd

def change_dtype(df, exclude, inplace=False):
    """
    Converts columns of type 'object' to numeric data types in a pandas DataFrame, excluding specified columns.

    Parameters:
        df (pandas.DataFrame): The input DataFrame.
        exclude (list of str): The names of columns to exclude from the operation.
        inplace (bool): Whether to modify the input DataFrame directly. Default is False.

    Returns:
        pandas.DataFrame or None: If inplace=True, returns None. Otherwise, returns a new DataFrame with the modified columns.
    """
    if inplace:
        for col in df.columns:
            if col not in exclude and df[col].dtype == "object":
                df[col] = pd.to_numeric(df[col], errors="coerce")
    else:
        new_df = df.copy()
        for col in new_df.columns:
            if col not in exclude and new_df[col].dtype == "object":
                new_df[col] = pd.to_numeric(new_df[col], errors="coerce")
        return new_df


In [45]:
df1 = change_dtype(df, ['DateTime', 'Holiday'])


In [46]:
df1.dtypes

DateTime                   object
Holiday                    object
HolidayFlag                 int64
DayOfWeek                   int64
WeekOfYear                  int64
Day                         int64
Month                       int64
Year                        int64
PeriodOfDay                 int64
ForecastWindProduction    float64
SystemLoadEA              float64
SMPEA                     float64
ORKTemperature            float64
ORKWindspeed              float64
CO2Intensity              float64
ActualWindProduction      float64
SystemLoadEP2             float64
SMPEP2                    float64
dtype: object